#Views para Dashboard

In [0]:
USE CATALOG controlled_medication_data;
USE SCHEMA gold_layer;

## Relatorio Geral

In [0]:
CREATE OR REPLACE VIEW vw_relatorio AS
SELECT 
    DATE(dt.data_venda) as data_venda,
    loc.uf_venda,
    loc.municipio_venda,
    loc.capital,
    prod.principio_ativo,
    presc.conselho_prescritor,
    presc.uf_conselho_prescritor,
    prod.CID10,
    fato.qtd_vendida
FROM ft_vendas as fato
INNER JOIN dim_data as dt
ON dt.id_data = fato.id_data
INNER JOIN dim_localizacao_venda as loc
ON fato.id_localizacao = loc.id_localizacao
INNER JOIN dim_produto as prod
ON fato.id_produto = prod.id_produto
INNER JOIN dim_prescritor as presc
ON presc.id_prescritor = fato.id_prescritor


## Princípios ativos mais vendido

In [0]:
CREATE OR REPLACE VIEW vw_top_principios_mais_vendidos AS 
SELECT 
    SUM(fato.qtd_vendida) AS qtd_vendida_total,
    produto.principio_ativo
FROM ft_vendas as fato
INNER JOIN dim_produto as produto
ON fato.id_produto = produto.id_produto
GROUP BY produto.principio_ativo
ORDER BY qtd_vendida_total DESC
LIMIT 10


## Princípio ativo por cidade

In [0]:
CREATE OR REPLACE VIEW vw_principio_ativo_por_cidade AS
SELECT 
    produto.principio_ativo,
    localizacao.municipio_venda as Cidade,
    localizacao.uf_venda as Estado,
    DATE(data_venda.data_venda) as data_venda,
    SUM(fato.qtd_vendida) as qtd_vendida
FROM ft_vendas as fato
INNER JOIN dim_produto as produto
ON fato.id_produto = produto.id_produto
INNER JOIN dim_localizacao_venda as localizacao
ON fato.id_localizacao = localizacao.id_localizacao
INNER JOIN dim_data as data_venda
ON fato.id_data = data_venda.id_data
GROUP BY
    produto.principio_ativo,
    Estado,
    Cidade,
    data_venda
ORDER BY qtd_vendida DESC


## Quantidade vendida do princípio ativo temporal

In [0]:
CREATE OR REPLACE VIEW vw_qtd_vendida_principio_ativo_temporal AS
SELECT 
    SUM(ft.qtd_vendida),
    prod.principio_ativo,
    DATE(data.data_venda)
FROM ft_vendas as ft
INNER JOIN dim_produto as prod
ON ft.id_produto = prod.id_produto
INNER JOIN dim_data as data
ON ft.id_data = data.id_data
GROUP BY 
    prod.principio_ativo,
    DATE(data.data_venda)

## Vendas por cidade

In [0]:
CREATE OR REPLACE VIEW vw_vendas_por_cidade AS
SELECT
  SUM(ft.qtd_vendida) AS numero_vendas,
  localizacao.uf_venda,
  localizacao.municipio_venda,
  localizacao.latitude_municipio,
  localizacao.longitude_municipio
FROM ft_vendas AS ft
INNER JOIN dim_localizacao_venda AS localizacao
  ON ft.id_localizacao = localizacao.id_localizacao
GROUP BY 
  localizacao.uf_venda,
  localizacao.municipio_venda,
  localizacao.latitude_municipio,
  localizacao.longitude_municipio
ORDER BY numero_vendas DESC

## Vendas por UF

In [0]:
CREATE OR REPLACE VIEW vw_vendas_por_uf AS
SELECT 
  localizacao.uf_venda,
  localizacao.nome_uf,
  localizacao.latitude_uf,
  localizacao.longitude_uf,
  SUM(ft.qtd_vendida) AS qtd_vendas
FROM ft_vendas AS ft
INNER JOIN dim_localizacao_venda AS localizacao
  ON ft.id_localizacao = localizacao.id_localizacao
GROUP BY 
  localizacao.uf_venda,
  localizacao.nome_uf,
  localizacao.latitude_uf,
  localizacao.longitude_uf
ORDER BY qtd_vendas DESC;